In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir())

# Any results you write to the current directory are saved as output.

['.git', '.gitignore', '.ipynb_checkpoints', 'items.csv', 'item_categories.csv', 'Predict Future Sales.ipynb', 'README.md', 'sales_train.csv.gz', 'sample_submission.csv.gz', 'shops.csv', 'test.csv.gz']


Data fields
ID - an Id that represents a (Shop, Item) tuple within the test set  
shop_id - unique identifier of a shop  
item_id - unique identifier of a product  
item_category_id - unique identifier of item category  
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure  
item_price - current price of an item  
date - date in format dd/mm/yyyy  
date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
item_name - name of item  
shop_name - name of shop  
item_category_name - name of item category  

*sales_train.csv* - the training set. Daily historical data from January 2013 to October 2015.   
*test.csv* - the test set. You need to forecast the sales for these shops and products for November 2015.  
*sample_submission.csv* - a sample submission file in the correct format.  
-- Item id column followed by item count for Nov 2015  
*items.csv* - supplemental information about the items/products.  
*item_categories.csv*  - supplemental information about the items categories.  
*shops.csv* - supplemental information about the shops.

In [7]:
sales_train = pd.read_csv('sales_train.csv.gz')
test = pd.read_csv('test.csv.gz')
sample_submission = pd.read_csv('sample_submission.csv.gz')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
all_csvs = [sales_train, test, sample_submission, items, item_categories, shops]

In [8]:
for file in all_csvs:
    print(file.shape)

(2935849, 6)
(214200, 3)
(214200, 2)
(22170, 3)
(84, 2)
(60, 2)


In [14]:
shops.head(2)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1


Simply the names of shops and shop_ids in the shops DF. Nothing for our machine learning there, obviously (at least for the work we'll be doing).

In [15]:
item_categories.head(2)

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1


item_categories.head(2)

Real shame I don't know Russian. Again, only names here. No descriptions or useful information for us. Perhaps if it were English we could add more categorization based on item names.

In [16]:
items.head(4)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40


In [11]:
items.shape

(22170, 3)

In [9]:
items.item_id.nunique()

22170

In [12]:
items.item_category_id.nunique()

84

As one might suspect, unique items can share category markers with other unique items. This could factor into features that look into seasonal purchases of categorical items. (E.g. It might be common to buy firewood and soup in winter months.) All items are unique, but there are only 84 categories.

In [13]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
